# Algoritmo de Monte Carlo Tree Search (MCTS)
MCTS es un algoritmo que combina la búsqueda de árbol con simulaciones aleatorias para evaluar las decisiones. Es ampliamente utilizado en juegos de estrategia y en juegos con información imperfecta. Funciona en cuatro fases principales:

1. **Selección:** Navega por el árbol de búsqueda hasta llegar a un nodo que no está completamente expandido.
2. **Expansión:** Expande el nodo seleccionado añadiendo uno o más nodos hijos.
3. **Simulación:** Realiza una simulación aleatoria (juego completo) desde el nuevo nodo para estimar el resultado.
4. **Retropropagación:** Actualiza el árbol de búsqueda con el resultado de la simulación.

## Implementación del MCTS para Triki
Aquí está una explicación paso a paso y un ejemplo de cómo implementar MCTS para el juego de Triki en Python:

1. **Definir el Estado del Juego**
Primero, necesitamos una clase para representar el estado del tablero y gestionar los movimientos.

In [ ]:
class Nodo:
    def __init__(self, tablero, jugador, movimiento=None):
        self.tablero = tablero
        self.jugador = jugador
        self.movimiento = movimiento
        self.hijos = {}
        self.visitas = 0
        self.puntuacion = 0

2. **Funciones para el Algoritmo MCTS**
Ahora, implementamos las funciones principales para MCTS: selección, expansión, simulación y retropropagación.

In [ ]:
import random

def movimientos_posibles(tablero):
    return [i for i, x in enumerate(tablero) if x == ' ']

def copiar_tablero(tablero):
    return tablero[:]

def simular(tablero, jugador):
    # Función que simula un juego aleatorio hasta el final
    while True:
        posibles = movimientos_posibles(tablero)
        if not posibles:
            return 0  # Empate
        movimiento = random.choice(posibles)
        tablero[movimiento] = jugador
        if evaluar_tablero(tablero) != 0:
            return 1 if jugador == 'X' else -1
        jugador = 'O' if jugador == 'X' else 'X'

def seleccionar(nodo):
    while nodo.hijos:
        mejor_puntuacion = float('-inf')
        mejor_hijo = None
        for hijo in nodo.hijos.values():
            ucb1 = (hijo.puntuacion / (hijo.visitas + 1)) + (2 * ((2 * (nodo.visitas + 1)) ** 0.5) / (hijo.visitas + 1))
            if ucb1 > mejor_puntuacion:
                mejor_puntuacion = ucb1
                mejor_hijo = hijo
        nodo = mejor_hijo
    return nodo

def expandir(nodo):
    posibles = movimientos_posibles(nodo.tablero)
    for movimiento in posibles:
        if movimiento not in nodo.hijos:
            nuevo_tablero = copiar_tablero(nodo.tablero)
            nuevo_tablero[movimiento] = nodo.jugador
            nuevo_jugador = 'O' if nodo.jugador == 'X' else 'X'
            nodo.hijos[movimiento] = Nodo(nuevo_tablero, nuevo_jugador, movimiento)
    return random.choice(list(nodo.hijos.values()))

def retropropagar(nodo, resultado):
    while nodo:
        nodo.visitas += 1
        nodo.puntuacion += resultado
        nodo = nodo.padre

def mcts(tablero, jugador, iteraciones=1000):
    raiz = Nodo(tablero, jugador)
    for _ in range(iteraciones):
        nodo_seleccionado = seleccionar(raiz)
        nodo_expandido = expandir(nodo_seleccionado)
        resultado = simular(nodo_expandido.tablero, nodo_expandido.jugador)
        retropropagar(nodo_expandido, resultado)

    mejor_movimiento = max(raiz.hijos.values(), key=lambda x: x.visitas).movimiento
    return mejor_movimiento


3. **Función para Evaluar el Tablero**
Utilizamos la misma función de evaluación para determinar el resultado del juego:

In [ ]:
def evaluar_tablero(tablero):
    lineas_ganadoras = [
        [tablero[0], tablero[1], tablero[2]], # fila 1
        [tablero[3], tablero[4], tablero[5]], # fila 2
        [tablero[6], tablero[7], tablero[8]], # fila 3
        [tablero[0], tablero[3], tablero[6]], # columna 1
        [tablero[1], tablero[4], tablero[7]], # columna 2
        [tablero[2], tablero[5], tablero[8]], # columna 3
        [tablero[0], tablero[4], tablero[8]], # diagonal 1
        [tablero[2], tablero[4], tablero[6]]  # diagonal 2
    ]

    for linea in lineas_ganadoras:
        if linea == ['X', 'X', 'X']:
            return 10
        elif linea == ['O', 'O', 'O']:
            return -10

    return 0


4. **Implementación del Juego**

Finalmente, integramos el MCTS en la función del juego.

In [ ]:
def imprimir_tablero(tablero):
    print(f"{tablero[0]} | {tablero[1]} | {tablero[2]}")
    print("--|---|--")
    print(f"{tablero[3]} | {tablero[4]} | {tablero[5]}")
    print("--|---|--")
    print(f"{tablero[6]} | {tablero[7]} | {tablero[8]}")

def juego_triki_mcts():
    tablero = [' '] * 9
    jugador_actual = 'X'

    while True:
        imprimir_tablero(tablero)

        if ' ' not in tablero:
            print("¡Empate!")
            break

        if jugador_actual == 'X':
            movimiento = mcts(tablero, 'X')
        else:
            movimiento = int(input("Ingrese su movimiento (0-8): "))

        if tablero[movimiento] == ' ':
            tablero[movimiento] = jugador_actual
        else:
            print("Movimiento inválido, intente de nuevo.")
            continue

        puntuacion = evaluar_tablero(tablero)
        if puntuacion == 10:
            imprimir_tablero(tablero)
            print("¡Jugador X gana!")
            break
        elif puntuacion == -10:
            imprimir_tablero(tablero)
            print("¡Jugador O gana!")
            break

        jugador_actual = 'O' if jugador_actual == 'X' else 'X'

juego_triki_mcts()


## Explicación
1. **Selección:** Navega por el árbol de búsqueda hasta encontrar un nodo no expandido.
2. **Expansión:** Añade nuevos nodos hijos al nodo seleccionado.
3. **Simulación:** Realiza un juego aleatorio desde el nuevo nodo para estimar el resultado.
4. **Retropropagación:** Actualiza la información en el árbol de búsqueda en función del resultado de la simulación.